In [89]:
# import re
#
# import numpy as np
# import numpy as np
# from pdex_json import create_df, stat_calc, cpms_calc, cpms
# import pandas as pd
# import math
#
#
# dex_df=create_df("./data/pdex_go.json")
# data = pd.read_csv("./data/poke_genie_export 3.csv", index_col = 0)
# data['Form'] = data['Form'].mask(data['Form'].fillna("Normal").str.contains('Normal',regex=True))
# dex_df.drop_duplicates(subset = ['dexNr','form','at','de','hp','type_1','type_2'],keep = 'first',ignore_index = True, inplace = True)
# # moves_df = dex_df['fast_move'].explode(ignore_index = True).to_frame()
# moves_df = dex_df['fast_move'].explode().to_frame().apply(lambda it: [list(move_dict(mov).values()) for na,mov in it['fast_move'].items()][0] if it['fast_move'] else [1,2,3,4,5,6], axis=1, result_type = 'expand').drop_duplicates()
# moves_df
# # moves_dict.update(*mo)
#     # moves_dict = moves_dict.update() | ent

NameError: name 'move_dict' is not defined

In [265]:
import pandas as pd
import requests
import json

# with open("./data/pdex_go.json", "r") as f:
#     data = json.loads(f.read())
url = 'https://raw.githubusercontent.com/Biuni/PokemonGO-Pokedex/master/pokedex.json'
# r = requests.get(url)
# data = json.loads(r.text)['pokemon']
# df = pd.read_json(json.dumps(data))
data = pd.read_json(url)

df = pd.DataFrame(data['pokemon'].apply(pd.Series))
df = pd.concat([df.drop(['next_evolution', 'prev_evolution'], axis=1), df['next_evolution'].apply(pd.Series).rename({0:'evo_1',1:'evo_2',2:'evo_3'},axis=1, errors='ignore')], axis=1)
df = pd.concat([df.drop(['img', 'type', 'weaknesses'], axis=1), df['type'].apply(pd.Series).rename({0:'primary_type',1:'sec_type'},axis=1)], axis=1)
df['evolutions'].str.get(0)
# df = pd.concat([df.drop(['move'], axis=1), df['moves'].apply(pd.Series)], axis=1)

# print(df[['name', 'id', 'height', 'weight', 'base-attack', 'base-defense', 'base-stamina', 'fast_moves', 'special_moves']])


KeyError: 'evolutions'

In [88]:
import pandas as pd
import requests
import json
dict_name_lookup = lambda x: x.get('names',x).get('English')

def create_mut_idx(x,y):
    x = x.drop(['combat'],axis=1)
    x[['type','names']] = x[['type','names']].applymap(dict_name_lookup, na_action = 'ignore')
    # li_y = [y]*x.shape[1]
    # x.columns.set_levels()
    x.drop_duplicates(subset='id', inplace=True)
    m_idx = pd.MultiIndex.from_product([[y], x['id'].values], names=['cat', 'id'])
    return x.drop(['id'],axis=1).set_index(m_idx,drop = True)
def combine_df(x):
    _ = pd.concat([x[y].dropna().explode().apply(pd.Series).pipe(create_mut_idx, y) for y in x.columns],axis=0)
    # _['type'] = _[].map(dict_name_lookup)
    # mi = _.columns.get_loc_level(['type','names'], level='cols')
    # _[['type','names']] = _[['type','names']].applymap(dict_name_lookup, na_action = 'ignore')
    return _.drop_duplicates()
    # return _.drop_duplicates().set_index(['cat','id']).sort_index(level=0, ascending=True, sort_remaining=True)

with open("./data/pdex_go.json", "r") as f:
    df = pd.read_json(f)
df = pd.concat([df, df['regionForms'].mask(df['regionForms'].apply(len)==0).dropna().map(pd.Series,na_action = 'ignore').explode().apply(pd.Series)], ignore_index = True, axis=0).sort_values(by=['dexNr','formId']).reset_index(drop=True)
# df['names'] = df['names'].str.get('English')
df[['names','primaryType','secondaryType']] = df[['names','primaryType','secondaryType']].applymap(dict_name_lookup, na_action = 'ignore')
df = pd.concat([df.drop(['stats'], axis=1), df['stats'].apply(pd.Series)], axis=1)
df['evolutions'] = df['evolutions'].str.get(0).str.get('id')
moves_cols = df.columns[df.columns.str.contains('Moves')].to_list()
moves_df = df[moves_cols].mask(df[moves_cols].applymap(len)==0).applymap(lambda x: list(x.values()), na_action = 'ignore').pipe(combine_df)
df[moves_cols] = df[moves_cols].mask(df[moves_cols].applymap(len)==0).applymap(lambda x: [y for y in x],na_action = 'ignore')
df['megaEvolutions'] = df['megaEvolutions'].str.join(sep=", ").str.rstrip(", ").mask(df['megaEvolutions'].apply(len)==0)#.apply(lambda x: ",".join(list(x.keys())))
df
# df['m'].mask

,id,formId,dexNr,generation,names,primaryType,secondaryType,pokemonClass,quickMoves,cinematicMoves,...,eliteCinematicMoves,assets,regionForms,evolutions,hasMegaEvolution,megaEvolutions,assetForms,stamina,attack,defense
0,BULBASAUR,BULBASAUR,1,1,Bulbasaur,Grass,Poison,None,"[VINE_WHIP_FAST, TACKLE_FAST]","[SLUDGE_BOMB, SEED_BOMB, POWER_WHIP]",...,NaN,{'image': 'https://raw.githubusercontent.com/P...,[],IVYSAUR,False,NaN,"[{'form': None, 'costume': 'JAN_2020_NOEVOLVE'...",128.0,118.0,111.0
1,IVYSAUR,IVYSAUR,2,1,Ivysaur,Grass,Poison,None,"[RAZOR_LEAF_FAST, VINE_WHIP_FAST]","[SLUDGE_BOMB, SOLAR_BEAM, POWER_WHIP]",...,NaN,{'image': 'https://raw.githubusercontent.com/P...,[],VENUSAUR,False,NaN,"[{'form': None, 'costume': None, 'isFemale': F...",155.0,151.0,143.0
2,VENUSAUR,VENUSAUR,3,1,Venusaur,Grass,Poison,None,"[RAZOR_LEAF_FAST, VINE_WHIP_FAST]","[SLUDGE_BOMB, PETAL_BLIZZARD, SOLAR_BEAM]",...,[FRENZY_PLANT],{'image': 'https://raw.githubusercontent.com/P...,[],NaN,True,VENUSAUR_MEGA,"[{'form': 'COPY_2019', 'costume': None, 'isFem...",190.0,198.0,189.0
3,CHARMANDER,CHARMANDER,4,1,Charmander,Fire,None,None,"[EMBER_FAST, SCRATCH_FAST]","[FLAME_CHARGE, FLAME_BURST, FLAMETHROWER]",...,NaN,{'image': 'https://raw.githubusercontent.com/P...,[],CHARMELEON,False,NaN,"[{'form': None, 'costume': 'JAN_2020_NOEVOLVE'...",118.0,116.0,93.0
4,CHARMELEON,CHARMELEON,5,1,Charmeleon,Fire,None,None,"[EMBER_FAST, FIRE_FANG_FAST]","[FIRE_PUNCH, FLAME_BURST, FLAMETHROWER]",...,NaN,{'image': 'https://raw.githubusercontent.com/P...,[],CHARIZARD,False,NaN,"[{'form': None, 'costume': 'JAN_2020_NOEVOLVE'...",151.0,158.0,126.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1196,URSALUNA,URSALUNA,901,9,Ursaluna,Ground,Normal,None,"[TACKLE_FAST, ROCK_SMASH_FAST]","[FIRE_PUNCH, THUNDER_PUNCH, AERIAL_ACE, ICE_PU...",...,[HIGH_HORSEPOWER],{'image': 'https://raw.githubusercontent.com/P...,[],NaN,False,NaN,"[{'form': None, 'costume': None, 'isFemale': F...",277.0,243.0,181.0
1197,SNEASLER,SNEASLER,903,9,Sneasler,Fighting,Poison,None,"[ROCK_SMASH_FAST, POISON_JAB_FAST, SHADOW_CLAW...","[AERIAL_ACE, CLOSE_COMBAT, X_SCISSOR]",...,NaN,{'image': 'https://raw.githubusercontent.com/P...,[],NaN,False,NaN,"[{'form': None, 'costume': None, 'isFemale': F...",190.0,259.0,158.0
1198,OVERQWIL,OVERQWIL,904,9,Overqwil,Dark,Poison,None,"[POISON_STING_FAST, POISON_JAB_FAST]","[AQUA_TAIL, ICE_BEAM, SLUDGE_BOMB, DARK_PULSE,...",...,NaN,{'image': 'https://raw.githubusercontent.com/P...,[],NaN,False,NaN,"[{'form': None, 'costume': None, 'isFemale': F...",198.0,222.0,171.0
1199,GIMMIGHOUL,GIMMIGHOUL,999,9,Gimmighoul,Ghost,None,None,[ASTONISH_FAST],[SHADOW_BALL],...,NaN,{'image': 'https://raw.githubusercontent.com/P...,[],GHOLDENGO,False,NaN,"[{'form': None, 'costume': None, 'isFemale': F...",128.0,140.0,76.0


In [80]:
from functools import reduce
temp_li = [*moves_cols]
temp_li = dict(zip(temp_li[:1],temp_li[:1]))
# temp_li
list(temp_li.keys()).pop() if len(temp_li.keys())==1 else ",".join(temp_li)
# ",".join(list(temp_li[:1])).rstrip(",")
# df['megaEvolutions'].mask(df['megaEvolutions'].apply(len)==0).dropna().apply(lambda x: ",".join(li) if len(li := list(x.keys()))>1 else li[0])
# df['primaryType'].apply(pd.Series)['names'].get('English')
# df['primaryType'] =
# di_t = df['primaryType'].map(lambda x: x['names']['English']).iat[0]


'quickMoves'

In [53]:
df

,id,formId,dexNr,generation,names,primaryType,secondaryType,pokemonClass,quickMoves,cinematicMoves,eliteQuickMoves,eliteCinematicMoves,assets,evolutions,hasMegaEvolution,megaEvolutions,assetForms,stamina,attack,defense
0,BULBASAUR,BULBASAUR,1,1,Bulbasaur,Grass,Poison,None,"[VINE_WHIP_FAST, TACKLE_FAST]","[SLUDGE_BOMB, SEED_BOMB, POWER_WHIP]",[],[],{'image': 'https://raw.githubusercontent.com/P...,IVYSAUR,False,[],"[{'form': None, 'costume': 'JAN_2020_NOEVOLVE'...",128.0,118.0,111.0
1,IVYSAUR,IVYSAUR,2,1,Ivysaur,Grass,Poison,None,"[RAZOR_LEAF_FAST, VINE_WHIP_FAST]","[SLUDGE_BOMB, SOLAR_BEAM, POWER_WHIP]",[],[],{'image': 'https://raw.githubusercontent.com/P...,VENUSAUR,False,[],"[{'form': None, 'costume': None, 'isFemale': F...",155.0,151.0,143.0
2,VENUSAUR,VENUSAUR,3,1,Venusaur,Grass,Poison,None,"[RAZOR_LEAF_FAST, VINE_WHIP_FAST]","[SLUDGE_BOMB, PETAL_BLIZZARD, SOLAR_BEAM]",[],[FRENZY_PLANT],{'image': 'https://raw.githubusercontent.com/P...,NaN,True,"{'VENUSAUR_MEGA': {'id': 'VENUSAUR_MEGA', 'nam...","[{'form': 'COPY_2019', 'costume': None, 'isFem...",190.0,198.0,189.0
3,CHARMANDER,CHARMANDER,4,1,Charmander,Fire,None,None,"[EMBER_FAST, SCRATCH_FAST]","[FLAME_CHARGE, FLAME_BURST, FLAMETHROWER]",[],[],{'image': 'https://raw.githubusercontent.com/P...,CHARMELEON,False,[],"[{'form': None, 'costume': 'JAN_2020_NOEVOLVE'...",118.0,116.0,93.0
4,CHARMELEON,CHARMELEON,5,1,Charmeleon,Fire,None,None,"[EMBER_FAST, FIRE_FANG_FAST]","[FIRE_PUNCH, FLAME_BURST, FLAMETHROWER]",[SCRATCH_FAST],[],{'image': 'https://raw.githubusercontent.com/P...,CHARIZARD,False,[],"[{'form': None, 'costume': 'JAN_2020_NOEVOLVE'...",151.0,158.0,126.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1196,URSALUNA,URSALUNA,901,9,Ursaluna,Ground,Normal,None,"[TACKLE_FAST, ROCK_SMASH_FAST]","[FIRE_PUNCH, THUNDER_PUNCH, AERIAL_ACE, ICE_PU...",[],[HIGH_HORSEPOWER],{'image': 'https://raw.githubusercontent.com/P...,NaN,False,[],"[{'form': None, 'costume': None, 'isFemale': F...",277.0,243.0,181.0
1197,SNEASLER,SNEASLER,903,9,Sneasler,Fighting,Poison,None,"[ROCK_SMASH_FAST, POISON_JAB_FAST, SHADOW_CLAW...","[AERIAL_ACE, CLOSE_COMBAT, X_SCISSOR]",[],[],{'image': 'https://raw.githubusercontent.com/P...,NaN,False,[],"[{'form': None, 'costume': None, 'isFemale': F...",190.0,259.0,158.0
1198,OVERQWIL,OVERQWIL,904,9,Overqwil,Dark,Poison,None,"[POISON_STING_FAST, POISON_JAB_FAST]","[AQUA_TAIL, ICE_BEAM, SLUDGE_BOMB, DARK_PULSE,...",[],[],{'image': 'https://raw.githubusercontent.com/P...,NaN,False,[],"[{'form': None, 'costume': None, 'isFemale': F...",198.0,222.0,171.0
1199,GIMMIGHOUL,GIMMIGHOUL,999,9,Gimmighoul,Ghost,None,None,[ASTONISH_FAST],[SHADOW_BALL],[],[],{'image': 'https://raw.githubusercontent.com/P...,GHOLDENGO,False,[],"[{'form': None, 'costume': None, 'isFemale': F...",128.0,140.0,76.0


In [348]:
def move_dict(_mov):
    _mov['names'] = _mov['names']['English'] if isinstance(_mov['names'],dict) else _mov['names']
    _mov['type'] = _mov['type']['names']['English'] if isinstance(_mov['type'],dict) else _mov['type']
    _mov = _mov | _mov['combat']
    [_mov.pop(col, None) for col in ['id','combat','buffs']]
    return _mov

# moves_dict = {}
moves_dict = {na:move_dict(mov) for ent in moves_df for na,mov in ent.items()}
moves_df = pd.DataFrame.from_dict(moves_dict, orient='index')
moves_df.columns
# moves_dict


NameError: name 'moves_df' is not defined

In [127]:
moves_df

,power,energy,durationMs,type,names,turns
Vine Whip,5,8,600,Grass,Vine Whip,2
Tackle,3,3,500,Normal,Tackle,1
Razor Leaf,10,4,1000,Grass,Razor Leaf,2
Ember,7,6,1000,Fire,Ember,2
Scratch,4,2,500,Normal,Scratch,1
...,...,...,...,...,...,...
Smack Down,12,8,1200,Rock,Smack Down,3
Leafage,6,7,700,Grass,Leafage,2
Gust,16,12,2000,Flying,Gust,4
Magical Leaf,10,10,1400,Grass,Magical Leaf,3


In [285]:

def find_pok(r):
    try:
        pok_lookup = (dex_df.dexNr.astype('int64')==int(r['Pokemon Number']))
        pok_df = dex_df.loc[pok_lookup,['names','form','at','de','hp']]
        if isinstance(r['Form'],str):
            pok_df = pok_df.loc[pok_df.form==r['Form'],['at','de','hp']]
        else:
            pok_df = pok_df.loc[pok_df.form.isna(),['at','de','hp']]
        return pok_df.take([0]).squeeze(axis = 0)
    except ValueError as err:
        raise err


# data['lvl_m'] = data['Level Min'].apply(lambda x: cpms_calc[x])

data[['c_at','c_de','c_hp']] = data.apply(find_pok, axis=1)




In [286]:
def calcCPBackend(r):
    # if verbose:
        # print (atk * math.pow(defe, 0.5) * math.pow(sta, 0.5) * math.pow(cpm, 2) / 10)
    try:
        defe = r['c_de']+int(r['Def IV'])
        atk = r['c_at']+r['Atk IV']
        sta = r['c_hp']+r['Sta IV']
        # print(atk, defe, sta)
        return int(max(10, math.floor(atk * math.pow(defe, 0.5) * math.pow(sta, 0.5) * math.pow(cpms[math.floor((r['Level Min']-1)*2)], 2) / 10)))
    except ValueError as err:
        # print (r)
        return 0

def calcHP(r):
    try:
        mul = cpms[math.floor((r['Level Min']-1)*2)]
        sta = r['c_hp']+r['Sta IV']
        return int(max(10, math.floor(mul * sta)))
    except ValueError:
        return 0



In [287]:
data['cp_calc'] = data.apply(calcCPBackend, axis = 1)
data['hp_calc'] = data.apply(calcHP, axis = 1)


In [279]:
data['Level Min'].apply(lambda r: cpms[math.floor((r-1)*2)])

Index
1      0.517394
2      0.517394
3      0.640653
4      0.290250
5      0.674582
         ...   
820    0.731700
821    0.719399
822    0.375236
823    0.462798
824    0.517394
Name: Level Min, Length: 824, dtype: float64

In [525]:


def stats_pvp(at_iv = 7, de_iv = 15, hp_iv = 15, poke_id = 2, level = 20):
    mul_stat = cpms[math.floor((level-1)*2)]
    de_stat = (dex_df.iloc[poke_id,9]+de_iv)
    hp_stat = (dex_df.iloc[poke_id,10]+hp_iv)
    de = math.pow(de_stat, 0.5)
    hp = math.pow(hp_stat, 0.5)
    base_at = dex_df.iloc[poke_id,8]
    l_start = (level-1)*2
    lvl_cp = 0
    stat_prev = 1000
    cp = 0
    lvl_stat = 0
    for lvl in range(l_start,len(cpms),1):
        lvl_stat_prev = lvl_stat
        ret_lvl = (lvl/2)+1
        mul_stat = cpms[lvl]
        # at_ret = n
        for n in range (at_iv, 16):
            at = base_at+n
            cp1 = (at*de*hp*math.pow(mul_stat,2))/10
            at_max = n
            if cp1 > 1500 or cp1 == lvl_cp:
                lvl_stat_prev = lvl_stat
                # at_max = at_ret
                break
            # cp = cp1
            lvl_stat_prev = lvl_stat
            lvl_stat = ((at*mul_stat)*(de_stat*mul_stat)*(hp_stat*mul_stat))/1000
            at_max = n
            cp = cp1
        print((lvl/2)+1, cp, lvl_stat, lvl_stat_prev)
        if lvl_stat < lvl_stat_prev:
            lvl_stat_prev = lvl_stat
            print((lvl/2)+1, cp, lvl_stat, lvl_stat_prev)
            break
        if cp > 1500 or cp == lvl_cp:
            break
        # print((lvl/2)+1, cp, lvl_stat, lvl_stat_prev)
        lvl_cp = cp
    # at_iv = n
    # return (at_iv,de_iv, hp_iv, level, cp, ((at*mul_stat)*(de_stat*mul_stat)*(
    return (at_max,de_iv, hp_iv, ret_lvl, cp, ((at*mul_stat)*(de_stat*mul_stat)*(hp_stat*mul_stat))/1000)

stats_pvp(0,15,15,2,15)
# int(max(10, math.floor(at * math.pow(de, 0.5) * math.pow(sta, 0.5) * math.pow(cpms[math.floor((r['Level Min']-1)*2)], 2) / 10)))

15.0 1166.0422969261565 1233.7514094197866 1227.9591492816655
15.5 1204.892122406224 1295.9207771668189 1289.8366420627492
16.0 1243.7419478862907 1359.100676185623 1352.7199218373337
16.5 1282.5917511887376 1423.2751577737083 1416.5931147794654
17.0 1321.4415544911838 1488.4290747822697 1481.4411448537144
17.5 1360.2914692588465 1554.5481101877137 1547.2497622525605
18.0 1399.1413840265107 1621.6181965067356 1614.0049655372202
18.5 1437.9911812012451 1689.6258270108763 1681.6933113911068
19.0 1476.8409783759741 1758.5584467636793 1750.302303821127
19.5 1494.3430301935055 1802.6516616422848 1802.6516616422848
20.0 1496.1540958576547 1827.8205904119552 1827.8205904119552
20.5 1496.1652037655901 1850.5479152419377 1850.5479152419377
21.0 1496.1652037655901 1850.5479152419377 1850.5479152419377


(0, 15, 15, 21.0, 1496.1652037655901, 1899.4955733120307)

In [295]:
|math.pow((dex_df.iloc[2,9]+15), 0.5)


14.2828568570857

In [416]:
at = cpms_calc[20.5]*(dex_df.iloc[2,8]+2)
de = cpms_calc[20.5]*(dex_df.iloc[2,9]+15)
hp = cpms_calc[20.5]*(dex_df.iloc[2,10]+15)
(at*de*hp)/1000


1850.5479152419377

In [275]:
level = 1.5
level += 0.5
math.floor((level-1) * 2)


2

In [259]:
def calcPVPBackend(r):
    # if verbose:
    # print (atk * math.pow(defe, 0.5) * math.pow(sta, 0.5) * math.pow(cpm, 2) / 10)
    try:
        defe = math.pow(r['lvl_m'],2)*(r['c_de']+int(r['Def IV']))
        atk = math.pow(r['lvl_m'],2)*(r['c_at']+r['Atk IV'])
        sta = math.pow(r['lvl_m'],2)*(r['c_hp']+r['Sta IV'])
        # print(atk, defe, sta)
        return int(atk * defe * sta)
    except ValueError as err:
        # print (r)
        return 0

In [260]:
data.apply(calcPVPBackend, axis = 1)


Index
1        40084
2       160077
3       127241
4         1088
5       579149
        ...   
820    1131349
821    1451917
822      26618
823      61095
824      39265
Length: 824, dtype: int64